# SmartPave Analytics: Feature Engineering

## Overview
This notebook creates advanced features for machine learning models to predict pavement degradation and optimize maintenance costs.

## Objectives
- Create time-based features
- Engineer traffic impact features
- Develop weather impact metrics
- Create maintenance history features
- Prepare data for ML modeling


In [ ]:
# Load data from Snowflake and create features
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Connect to Snowflake
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Load pavement condition data
df = session.sql("SELECT * FROM pavement_condition").to_pandas()
df['date'] = pd.to_datetime(df['date'])

print(f"Loaded dataset: {len(df):,} records")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"Unique segments: {df['segment_id'].nunique():,}")

# Load maintenance data for feature engineering
maintenance_df = session.sql("SELECT * FROM maintenance_records").to_pandas()
maintenance_df['date'] = pd.to_datetime(maintenance_df['date'])
